In [20]:
import sys
!{sys.executable} -m pip install pulp

from pulp import *

item_list = ["Shirts", "Shorts", "Pants"]
num_items = len(item_list)
labour_list = [3, 2, 6] # in hours
cloth_list = [4, 3, 4] # in m^2
price_list = [12, 8, 15] # in $
cost_list = [6, 4, 8] # in $
machine_cost_list = [200, 150, 100] # in $

labour_max = 150 # hours
cloth_max = 160 # m^2

profit_list = [price_list[i] - cost_list[i] for i in range(num_items)]

problem = LpProblem("clothco", LpMaximize)

# Decision Variables

# Variables for the number of items
v_items = LpVariable.dicts("num_items", list(range(num_items)), 0, cat=LpInteger)
# Variables for machine rent use, binary on/off switch
v_supp = LpVariable.dicts("machine_rent", list(range(len(item_list), 2 * len(item_list) + 1)), 0, 1, cat=LpInteger)

# Add the dictionaries together.
v_items.update(v_supp)
v = v_items

# Constraints
c1 = lpSum(labour_list[i] * v[i] for i in range(num_items)) <= labour_max
c2 = lpSum(cloth_list[i] * v[i] for i in range(num_items)) <= cloth_max

c3 = v[0] <= 40 * v[3]
c4 = v[1] <= 160/3 * v[4]
c5 = v[2] <= 25 * v[5]

problem += c1
problem += c2
problem += c3
problem += c4
problem += c5

# Objective function
problem += lpSum(profit_list[i] * v[i] for i in range(num_items)) \
            - lpSum(machine_cost_list[j-num_items] * v[j] for j in range(3,6))

print(problem)

# solving
problem.solve()

print("ClothCo should make", end=' ')
for i in range(3):
    print(f"{v[i].varValue} {item_list[i]},", end=' ')

z = []
for j in range(3,6):
    if v[j].varValue == 1:
        z.append(item_list[j-3])
        
print("We are only using the machine/s to make", ','.join(z))

obj = sum(profit_list[i] * v[i].varValue for i in range(num_items)) \
            - sum(machine_cost_list[j-num_items] * v[j].varValue for j in range(3,6))

print(f"The profit will be ${obj}")


clothco:
MAXIMIZE
-200*machine_rent_3 + -150*machine_rent_4 + -100*machine_rent_5 + 6*num_items_0 + 4*num_items_1 + 7*num_items_2 + 0
SUBJECT TO
_C1: 3 num_items_0 + 2 num_items_1 + 6 num_items_2 <= 150

_C2: 4 num_items_0 + 3 num_items_1 + 4 num_items_2 <= 160

_C3: - 40 machine_rent_3 + num_items_0 <= 0

_C4: - 53.3333333333 machine_rent_4 + num_items_1 <= 0

_C5: - 25 machine_rent_5 + num_items_2 <= 0

VARIABLES
0 <= machine_rent_3 <= 1 Integer
0 <= machine_rent_4 <= 1 Integer
0 <= machine_rent_5 <= 1 Integer
0 <= num_items_0 Integer
0 <= num_items_1 Integer
0 <= num_items_2 Integer

ClothCo should make 0.0 Shirts, 0.0 Shorts, 25.0 Pants, We are only using the machine/s to make Pants
The profit will be $75.0
